## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [ ]:
!wget --no-check-certificate "https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_5340.zip"

In [ ]:
!unzip /content/kagglecatsanddogs_5340.zip

In [25]:
import keras
# from tensorflow.keras.datasets import cifar10
from tensorflow.keras.applications.resnet50 import ResNet50
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.optimizers import RMSprop, Adam

In [47]:
batch_size = 128 # batch 的大小，如果出現 OOM error，請降低這個值
num_classes = 10 # 類別的數量，Cifar 10 共有 10 個類別
epochs = 10 # 訓練的 epochs 數量

In [32]:
import os, cv2, glob
x = []
y = []
dict1 = {'Cat':0, 'Dog':1}
for folder in glob.glob('/content/PetImages/*') :
    print(folder,'讀取')
    label = folder.split('/')[-1]
    for f in os.listdir(folder):
        try:
            img = cv2.imread(os.path.join(folder, f))
            img = cv2.resize(img, dsize=(80,80))
            if img is not None:
                x.append(img)
                y.append(dict1[label])
        except:
            pass

print('完成')

/content/PetImages/Dog 讀取
/content/PetImages/Cat 讀取
完成


In [42]:
import numpy as np
from sklearn.model_selection import train_test_split
x = np.array(x)
y = np.array(y)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [46]:
# (x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = keras.utils.np_utils.to_categorical(y_train, num_classes)
y_test = keras.utils.np_utils.to_categorical(y_test, num_classes)

x_train shape: (19956, 80, 80, 3)
19956 train samples
4990 test samples


In [58]:
net = ResNet50(include_top=False, 
               weights='imagenet',
               input_tensor=None,
               input_shape=x_train.shape[1:])
x = net.output
x = Flatten()(x)
x = Dropout(0.25)(x)
output_layer = Dense(num_classes, activation='softmax', name='softmax')(x)  # 增加 Dense layer，以 softmax 產生個類別的機率值
model = Model(inputs=net.input, outputs=output_layer)  # 設定凍結與要進行訓練的網路層

for layer in model.layers[:2]:
    layer.trainable = False
for layer in model.layers[2:]:
    layer.trainable = True

In [ ]:
model.summary()

In [62]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [50]:
train_datagen = ImageDataGenerator(rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   channel_shift_range=10,
                                   horizontal_flip=True,
                                   fill_mode='nearest')

In [63]:
model.fit_generator(train_datagen.flow(x_train, y_train, batch_size),
                    steps_per_epoch=len(x_train)//batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  """


Epoch 1/10
155/155 [==============================] - 121s 342ms/step - loss: 1.3691 - accuracy: 0.5055 - val_loss: 181648.3281 - val_accuracy: 0.4906
Epoch 2/10
155/155 [==============================] - 50s 324ms/step - loss: 1.0834 - accuracy: 0.5072 - val_loss: 18.8452 - val_accuracy: 0.5044
Epoch 3/10
155/155 [==============================] - 50s 322ms/step - loss: 0.8574 - accuracy: 0.5088 - val_loss: 0.6975 - val_accuracy: 0.5094
Epoch 4/10
155/155 [==============================] - 51s 327ms/step - loss: 0.9376 - accuracy: 0.5098 - val_loss: 1390.4016 - val_accuracy: 0.4733
Epoch 5/10
155/155 [==============================] - 50s 320ms/step - loss: 1.0314 - accuracy: 0.5056 - val_loss: 1.1260 - val_accuracy: 0.4892
Epoch 6/10
155/155 [==============================] - 50s 321ms/step - loss: 0.8061 - accuracy: 0.4993 - val_loss: 0.6940 - val_accuracy: 0.4888
Epoch 7/10
155/155 [==============================] - 52s 336ms/step - loss: 0.7606 - accuracy: 0.5063 - val_loss: 0.693

KeyboardInterrupt: ignored

In [64]:
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

156/156 [==============================] - 4s 25ms/step - loss: 0.6991 - accuracy: 0.4912
Test loss: 0.6991181969642639
Test accuracy: 0.4911823570728302
